In [1]:
import pandas as pd
import glob

#print(glob.glob("/home/jovyan/*"))
allFiles = glob.glob("/home/jovyan/work/data/all_0000000000*")
print(allFiles)
data = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
data = pd.concat(list_)


['/home/jovyan/work/data/all_000000000013', '/home/jovyan/work/data/all_000000000014', '/home/jovyan/work/data/all_000000000012', '/home/jovyan/work/data/all_000000000008', '/home/jovyan/work/data/all_000000000001', '/home/jovyan/work/data/all_000000000006', '/home/jovyan/work/data/all_000000000007', '/home/jovyan/work/data/all_000000000000', '/home/jovyan/work/data/all_000000000009', '/home/jovyan/work/data/all_000000000010', '/home/jovyan/work/data/all_000000000011', '/home/jovyan/work/data/all_000000000005', '/home/jovyan/work/data/all_000000000002', '/home/jovyan/work/data/all_000000000003', '/home/jovyan/work/data/all_000000000004']


In [2]:
def filter_train_type(df=[],
                      train_types=[],
                      sum_types = False,
                      sum_columns=['train_count', 'delay', 'total_delay'],
                      aggs={'pressure': 'max',
                            'max_temperature': 'max',
                            'min_temperature': 'min',
                            'mean_temperature': 'mean',
                            'mean_dewpoint': 'mean',
                            'mean_humidity': 'mean',
                            'mean_winddirection': 'mean',
                            'mean_windspeed': 'mean',
                            'max_windspeedms': 'max',
                            'max_windgust': 'max',
                            'max_precipitation1h': 'max',
                            'max_snowdpeth': 'max',
                            'max_n': 'max',
                            'min_vis': 'min',
                            'min_clhb': 'min',
                            'max_precipitation3h': 'max',
                            'max_precipitation6h': 'max'}):
    """
    Filter traintypes from metadata

    train_types     : list
                      list of following options: [0,1,2,3]

                      train_types = {'K': 0,
                                     'L': 1,
                                     'T': 2,
                                     'M': 3}
    sum_types       : bool
                     if True, sum different train types together (default False)

    returns : np array, np array
              labels metadata, labels
    """

    if len(df) == 0:
        return df

    mask = df.loc[:,'train_type'].isin(train_types)
    filt_df = df[(mask)]

    if sum_types:
        d = {}
        for col in sum_columns:
            d[col] = ['sum']
        for col,method in aggs.items():
            d[col] = [method]
        if 'lat' in df:
            d['lat'] = ['max']
        if 'lon' in df:
            d['lon'] = ['max']

        filt_df = filt_df.groupby(['trainstation', 'time'], as_index=False).agg(d)

        filt_df.columns = filt_df.columns.droplevel(1)
        filt_df.drop_duplicates(['trainstation', 'time'], inplace=True)

    return filt_df

data_filt = filter_train_type(data, ['K', 'L'])

In [ ]:
print(data_filt)

In [3]:
data_filt.sort_values(by=['time', 'trainstation'], inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
data_filt_time = data_filt.loc[(data_filt['time'] >= '2010-01-01') & (data_filt['time'] <= '2010-03-31')]

In [8]:
data_filt_tiny = data_filt.loc[(data_filt['time'] >= '2010-01-01') & (data_filt['time'] <= '2010-01-10')]

In [7]:
print(data_filt_time)

                            time trainstation train_type  train_count  \
1287675  2010-01-01 00:00:00 UTC          HKH          L            1   
1283303  2010-01-01 00:00:00 UTC          HKH          K            1   
1284384  2010-01-01 00:00:00 UTC          HKH          L            2   
1296206  2010-01-01 00:00:00 UTC          HKI          K            1   
1296264  2010-01-01 00:00:00 UTC          HKI          L            1   
1296340  2010-01-01 00:00:00 UTC          HKI          L            3   
1280525  2010-01-01 00:00:00 UTC          HPL          L            2   
1278335  2010-01-01 00:00:00 UTC          HPL          L            1   
1283092  2010-01-01 00:00:00 UTC          HPL          L            3   
1287532  2010-01-01 00:00:00 UTC           HY          L            1   
1285833  2010-01-01 00:00:00 UTC           HY          L            1   
1258600  2010-01-01 00:00:00 UTC          JAL          K            1   
1287835  2010-01-01 00:00:00 UTC           JK      

In [26]:
data_filt.to_csv('/home/jovyan/work/data/all_filtered.csv', encoding='utf-8')

In [6]:
data_filt_time.to_csv('/home/jovyan/work/data/2010q1_filtered.csv', encoding='utf-8')

In [10]:
data_filt_tiny.to_csv('/home/jovyan/work/data/tiny_filtered.csv', encoding='utf-8')